In [1]:
import tensorflow as tf
from utils import load_class_names, output_boxes, draw_outputs, resize_image
from yolov3 import YOLOv3Net
import cv2
import time

#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
#tf.config.experimental.set_memory_growth(physical_devices[0], True)
times = []
model_size = (416, 416,3)
num_classes = 80
class_name = './data/coco.names'
max_output_size = 100
max_output_size_per_class= 20
iou_threshold = 0.5
confidence_threshold = 0.5


cfgfile = 'cfg/yolov3.cfg'
weightfile = 'weights/yolov3_weights.tf'
videopath = "./data/videos/traffic.mp4"


model = YOLOv3Net(cfgfile,model_size,num_classes)
model.load_weights(weightfile)
class_names = load_class_names(class_name)

#Reading video file

#specify the vidoe input.
# 0 means input from cam 0.
# For vidio, just change the 0 to video path
#cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture(videopath)

# obtaining frame size information
width = int(cap.get(3))
height = int(cap.get(4))
frame_size = (width,height)

#writing video
output = cv2.VideoWriter('output/detected_traffic.avi',cv2.VideoWriter_fourcc('M','J','P','G'),30,frame_size)

while True:
    start = time.time()
    ret,frame = cap.read()
    if not ret:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    
    resized_frame = tf.expand_dims(frame, 0)
    resized_frame = resize_image(resized_frame, (model_size[0],model_size[1]))
    

    pred = model.predict(resized_frame)

    boxes, scores, classes, nums = output_boxes(\
                pred, model_size,
                max_output_size=max_output_size,
                max_output_size_per_class=max_output_size_per_class,
                iou_threshold=iou_threshold,
                confidence_threshold=confidence_threshold)

    img = draw_outputs(frame, boxes, scores, classes, nums, class_names)
    output.write(img)
    window_name = 'detection'
    cv2.imshow(window_name ,img)
    stop = time.time()
    seconds = stop - start
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows() 
cap.release()
print('Detections have been performed successfully.')
    


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 416, 416, 3) 0                                            
__________________________________________________________________________________________________
tf_op_layer_truediv (TensorFlow [(None, 416, 416, 3) 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 416, 416, 32) 864         tf_op_layer_truediv[0][0]        
__________________________________________________________________________________________________
bnorm_0 (BatchNormalization)    (None, 416, 416, 32) 128         conv_0[0][0]                     
______________________________________________________________________________________________

Detections have been performed successfully.
